In [6]:
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN, MeanShift, MiniBatchKMeans, Birch, AffinityPropagation, AgglomerativeClustering
from sklearn import metrics

In [3]:
df_data = pd.read_csv("../../results_demo/T50/FFT/Group_0/fft-Group_0-T50.csv")
df_data

,id,target,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,...,p_246,p_247,p_248,p_249,p_250,p_251,p_252,p_253,p_254,p_255
0,sequence_53082,55.0,11394.6611,2325.0369,1266.0856,4881.6605,4945.7870,3392.7322,6681.9953,2752.2656,...,4796.0485,523.3252,1205.8106,2139.6097,3696.9292,5707.7871,1453.0932,7069.2878,2062.5873,4145.1232
1,sequence_19402,43.0,7281.9469,1953.9591,2672.5966,2594.4823,3238.9036,1183.7899,4512.5105,3776.9749,...,362.2150,1062.6668,4485.4121,940.0865,3309.1373,3557.0784,2146.7805,3998.3450,3889.5583,1372.5579
2,sequence_20054,49.0,5935.1587,4383.5804,931.2379,3493.8992,4508.2230,2470.0254,4816.7737,7111.1114,...,3536.8943,2094.6476,2734.4204,1540.7550,5101.0814,2879.6260,3969.4475,4712.8468,2212.5651,1801.7112
3,sequence_20048,39.8,6955.6274,1251.3568,3423.3544,2213.0241,4159.5633,1145.3681,7875.8760,4692.5329,...,3400.8646,2657.8203,3707.0671,1085.4381,5052.6596,2382.1370,2301.6946,2914.4836,2134.9467,2908.2101
4,sequence_20036,52.9,11760.8324,2067.0490,3067.9070,3822.7452,5883.5075,1585.7887,6515.4937,5969.5741,...,3650.8104,2020.8274,2693.0812,832.3258,5753.1930,3248.5301,3298.4924,4543.2944,1395.7901,1395.4314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,sequence_19347,58.9,10062.7221,3661.0614,1279.7899,4093.2332,4303.1550,4693.2341,5032.5104,4454.6462,...,3280.4593,1046.8846,2465.2036,3119.6391,4823.9983,6115.3716,718.8150,6582.5984,1699.3960,4358.5493
257,sequence_19379,50.3,11525.5793,3316.1664,2590.0420,3810.6741,3265.5993,4566.1141,5796.2374,4125.2626,...,2123.3688,2831.8437,2605.5751,978.0575,2862.2263,4015.3174,1399.6246,5459.0861,1950.2025,3779.2604
258,sequence_19348,46.7,9201.6412,4239.0843,1828.5824,5957.6869,3296.5241,5149.2625,5621.1634,2861.0159,...,901.5363,3015.5806,2531.5106,2866.4867,5043.6503,5163.1958,4137.0492,3224.9087,2514.0349,2739.0084
259,sequence_19380,48.1,10664.4984,4179.1677,2856.5334,5616.0164,2481.2805,4684.5143,6461.9872,2530.5632,...,736.9345,4103.2539,1465.9229,742.6825,3462.5478,3234.8363,2287.5811,2996.6131,1501.3620,2020.5958


In [8]:
def get_performance_clustering(data, labels):
    siluetas = metrics.silhouette_score(data, labels, metric='euclidean')
    calinski = metrics.calinski_harabasz_score(data, labels)
    davies = metrics.davies_bouldin_score(data, labels)

    return siluetas, calinski, davies

In [4]:
#ignoramos y removemos las columnas que no ocuparemos para el clustering
ignore_columns = pd.DataFrame()
ignore_columns['id'] = df_data['id']
ignore_columns['target'] = df_data['target']

df_data = df_data.drop(columns=['id', 'target'])

In [12]:
#exploramos KNN
matrix_result = []
for k in range(2, 30):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(df_data)
    siluetas, calinski, davies = get_performance_clustering(df_data, kmeans.labels_)
    row = ["k-means-{}".format(k), siluetas, calinski, davies]
    matrix_result.append(row)

df_explore = pd.DataFrame(matrix_result, columns=['strategy', 'siluetas', 'calinski', 'davies'])

In [17]:
#seleccionamos el mejor
import numpy as np

highest_siluetas = np.max(df_explore['siluetas'])
highest_calinski = np.max(df_explore['calinski'])

# filtramos
df_filter_by_siluetas = df_explore.loc[df_explore['siluetas'] >= highest_siluetas]
df_filter_by_calinski = df_explore.loc[df_explore['calinski'] >= highest_calinski]

df_concat = pd.concat([df_filter_by_siluetas, df_filter_by_calinski])
strategies = df_concat['strategy'].unique()
strategies


array(['k-means-2'], dtype=object)

In [19]:
# en este caso salio el mejor k-means=2
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(df_data)
ignore_columns['labels'] = kmeans.labels_

ignore_columns.to_csv("../../results_demo/T50/FFT/Group_0/unsupervised_clustering_sequences.csv")